<a href="https://colab.research.google.com/github/werlang/emolearn-ml-model/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence, plot_model, to_categorical
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, GRU, LSTM, Dropout, Conv1D, Conv2D, Conv3D, ConvLSTM2D, BatchNormalization, MaxPooling1D, MaxPooling2D, MaxPooling3D, GlobalAveragePooling2D, Flatten, Dense, Input, Add, Activation, AveragePooling3D, AveragePooling2D, ZeroPadding3D, Bidirectional, Concatenate
from tensorflow.keras.optimizers import Adam, SGD, Nadam
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, Callback, ModelCheckpoint
from tensorflow.keras.metrics import AUC
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import os, cv2
import datetime
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import math
from IPython.display import Image, display
from numba import cuda
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler 
import functools

# from imblearn.over_sampling import SMOTE
# sm = SMOTE(sampling_strategy=0.6666)
# X, y = sm.fit_resample(X, y)


drive_save_path = 'drive/My Drive/1NOSYNC/DT/checkpoint'
ident_name = 'final'
dir_name = '2022-8-8-17-18-19-final'
batch_size = 64
time_frames = 20
interval = 2
stride = 1
fold_step = 1
n_folds = 5

epoch = 62


def restart():
    cuda.select_device(0)
    cuda.close()


def start_colab():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    !pip install Keras-Applications
    # !pip install git+https://github.com/rcmalli/keras-vggface.git
    !pip install keras-tcn


def extract_data():
    !mkdir p2m
    !mkdir p2m/features

    # aligned faces extracted from openface
    print("COPYING ALIGNED FACES...")
    !unzip -n -q "drive/My Drive/1NOSYNC/DT/p2m_dataset/224/p2m_faces.zip" -d p2m/features
    print("COPYING OPENFACE FEATURES...")
    !unzip -n -q "drive/My Drive/1NOSYNC/DT/p2m_dataset/224/p2m_openface.zip" -d p2m/features
    print("COPYING LABELS...")
    !cp -r "drive/My Drive/1NOSYNC/DT/p2m_dataset/224/labels.csv" p2m

    print("P2M DONE")

    !mkdir daisee
    !mkdir daisee/features

    print("COPYING TRAIN SET...")
    !unzip -n -q "drive/My Drive/1NOSYNC/DT/daisee_aligned/Train.zip" -d daisee/features 
    print("COPYING VALIDATION SET...")
    !unzip -n -q "drive/My Drive/1NOSYNC/DT/daisee_aligned/Validation.zip" -d daisee/features 
    print("COPYING TEST SET...")
    !unzip -n -q "drive/My Drive/1NOSYNC/DT/daisee_aligned/Test.zip" -d daisee/features 
    print("COPYING OPENFACE FEATURES...")
    !unzip -n -q "drive/My Drive/Doutorado/Implementação/daisee_openface.zip" -d daisee/of_features
    print("COPYING LABELS...")
    !cp -r "drive/My Drive/Doutorado/Implementação/labels" daisee

    print("DAISEE DONE")


def create_folds_p2m(em):
    # ["ENGAGEMENT", "CONFUSION", "FRUSTRATION", "BOREDOM"]
    emotionList = ["engagement", "confusion", "frustraton", "boredom"]
    emotion = emotionList.index(em)
    print("Emotion: {}".format(emotionList[emotion]))

    num_classes = 2
    label_window = time_frames
    print("CREATING FOLDS...")
    subs = {}

    csv = pd.read_csv('p2m/labels.csv')
    Y = np.array(csv.iloc[:, 3 + emotion ])
    subjects = np.array(csv['SUBJECT'])
    clips = np.array(csv['CLIP'])
    source_frames = np.array(csv['SOURCE_FRAMES'])

    Y = to_categorical(Y, num_classes)    

    for i in range(len(subjects)):
        if not subjects[i] in subs:
            subs[subjects[i]] = [0 for x in range(num_classes)]
        subs[subjects[i]] = np.add(subs[subjects[i]], Y[i])

    # total = np.sum(Y, axis=0)
    # print(total, total / len(Y))
    # print(subs)

    folds = []
    fold_names = []
    for i in subs:
        if len(folds) < n_folds + 1:
            folds.append(subs[i])
            fold_names.append([i])
        else:
            summed = np.sum(folds, axis=1)
            index = list(summed).index(min(summed))
            folds[index] = np.add(folds[index], subs[i])
            fold_names[index].append(i)

    # print(folds)
    # print(fold_names)
    # print(np.sum(folds, axis=1))

    # print("GETTING FOLDS DATA...")
    # build frames array
    videos, labels, opface, xlabel = [], [], [], []
    for i in range(len(fold_names)):
        videos.append([])
        opface.append([])
        labels.append([])
        xlabel.append([])

    fps = 30
    skip = int(round(interval * fps / time_frames, 0))

    for Yi in range(len(Y)):
        print("\rCLIP {}/{}".format(Yi, len(Y)), end="")
        clip = clips[Yi]
        start_frame = source_frames[Yi].split("-")[0]
        subject = subjects[Yi]

        for f in range(n_folds + 1):
            if int(subject) in fold_names[f]:
                files, filesof = [], []
                
                of_path = "p2m/features/{:02}.csv".format(subject)

                for i in range(60):
                    image_path = "p2m/features/faces/{:02}.{:03}.{:02}.jpg".format(subject, clip, i)
                    
                    if os.path.isfile(image_path) and os.path.isfile(of_path):
                        files.append(image_path)
                        filesof.append("{}|{}".format(of_path, int(start_frame) + i))

                # append only the parts relative to the video section
                last = (len(files) - time_frames*skip) // skip
                rg = range(0, last + 1, np.max([1, int(stride * time_frames)]))
                for i in rg:
                    temp, tpof = [], []
                    for j in range(i*skip, (i+time_frames)*skip, skip):
                        temp.append(files[j])
                        tpof.append(filesof[j])
                    videos[f].append(temp)
                    opface[f].append(tpof)
                    labels[f].append(Y[Yi])

                    # get previous clip labels from same subject
                    xl = []
                    for l in range(Yi - 1, 0, -1):
                        sub2 = subjects[l]
                        if subject == sub2:
                            xl.append(Y[l])
                        if len(xl) == label_window:
                            break
                        
                    xl = [(xl[0] if len(xl) > 0 else np.array([0., 1.])) for x in range(label_window - len(xl))] + xl
                    xlabel[f].append(xl)


    print("\nDONE")
    return videos, opface, xlabel, labels


def create_folds_daisee(em):
    # ["Boredom", "Engagement", "Confusion", "Frustration"]
    emotionList = ["boredom", "engagement", "confusion", "frustraton"]
    emotion = emotionList.index(em)
    print("Emotion: {}".format(emotionList[emotion]))

    num_classes = 2
    label_window = time_frames
    print("CREATING FOLDS...")
    subs = {}

    Y = []
    clips = []
    # just concatenate all splits
    for split in ["Train", "Validation"]:
        csv_path = "daisee/labels/{}Labels.csv".format(split)
        csv = pd.read_csv(csv_path)
        C = np.array(csv['ClipID'])
        Yt = np.array(csv.iloc[:,1:])
        if len(Y) == 0:
            clips = C
            Y = Yt
        else:
            Y = np.concatenate((Y, Yt), axis=0)
            clips = np.concatenate((clips, C), axis=0)

    Y = to_categorical(np.array(Y[:,emotion]) // (4 // num_classes), num_classes)  

    for i in range(len(clips)):
        file_name = clips[i].split(".")[0]
        subject = int(file_name[:6])
        if not subject in subs:
            subs[subject] = [0 for x in range(num_classes)]
        subs[subject] = np.add(subs[subject], Y[i])

    # total = np.sum(Y, axis=0)
    # print(total, total / len(Y))
    # print(subs)

    folds = []
    fold_names = []
    for i in subs:
        if len(folds) < n_folds:
            folds.append(subs[i])
            fold_names.append([i])
        else:
            summed = np.sum(folds, axis=1)
            index = list(summed).index(min(summed))
            folds[index] = np.add(folds[index], subs[i])
            fold_names[index].append(i)

    # print(folds)
    # print(fold_names)
    # print(np.sum(folds, axis=1))

    # print("GETTING FOLDS DATA...")
    # build frames array
    videos, labels, opface, xlabel = [], [], [], []
    for i in range(len(fold_names)):
        videos.append([])
        opface.append([])
        labels.append([])
        xlabel.append([])

    fps = 15
    skip = int(round(interval * fps / time_frames, 0))

    split_start = []
    csv_path = "daisee/labels/TrainLabels.csv"
    csv = pd.read_csv(csv_path)
    split_start.append(len(csv))
    csv_path = "daisee/labels/ValidationLabels.csv"
    csv = pd.read_csv(csv_path)
    split_start.append(split_start[0] + len(csv))

    for Yi in range(len(clips)):
        print("\rCLIP {}/{}".format(Yi, len(clips)), end="")
        file_name = clips[Yi].split(".")[0]
        subject = file_name[:6]

        for f in range(n_folds):
            if int(subject) in fold_names[f]:
                # print("{}/{}".format(split, subject))
                split = 'Train' if Yi < split_start[0] else 'Validation'
                # get aligned faces file names into files array
                dir_path = "daisee/features/{}/{}/{}_aligned".format(split, subject, file_name)
                of_path = "daisee/of_features/{}/{}/{}.csv".format(split, subject, file_name)
                files, filesof = [], []
                
                # there are only 150 files in each folder
                for i in range(1,350):
                    image_path = "{}/frame_det_00_000{:03d}.jpg".format(dir_path, i)
                    if os.path.isfile(image_path) and os.path.isfile(of_path):
                        files.append(image_path)
                        filesof.append("{}|{}".format(of_path, i-1))

                # append only the parts relative to the video section
                last = (len(files) - time_frames*skip) // skip
                rg = range(0, last + 1, np.max([1, int(stride * time_frames)]))
                for i in rg:
                    temp, tpof = [], []
                    for j in range(i*skip, (i+time_frames)*skip, skip):
                        temp.append(files[j])
                        tpof.append(filesof[j])
                    videos[f].append(temp)
                    opface[f].append(tpof)
                    labels[f].append(Y[Yi])

                    # get previous clip labels from same subject
                    xl = []
                    for l in range(Yi - 1, 0, -1):
                        sub2 = clips[l].split(".")[0][:6]
                        if subject == sub2:
                            xl.append(Y[l])
                        if len(xl) == label_window:
                            break
                        
                    xl = [(xl[0] if len(xl) > 0 else np.array([0., 1.])) for x in range(label_window - len(xl))] + xl
                    xlabel[f].append(xl)


    # get test fold
    print("\nBUILDING TEST DATA")
    csv_path = "daisee/labels/TestLabels.csv"
    csv = pd.read_csv(csv_path)

    clips = np.array(csv['ClipID'])
    Y = to_categorical(np.array(csv.iloc[:, 1 + emotion]) // (4 // num_classes), num_classes)

    videos_test, labels_test, opface_test, xlabel_test = [], [], [], []

    for Yi in range(len(clips)):
        print("\rCLIP {}/{}".format(Yi, len(Y)), end="")
        file_name = clips[Yi].split(".")[0]
        subject = file_name[:6]

        dir_path = "daisee/features/Test/{}/{}_aligned".format(subject, file_name)
        of_path = "daisee/of_features/Test/{}/{}.csv".format(subject, file_name)

        files, filesof = [], []
        
        for i in range(1,350):
            image_path = "{}/frame_det_00_000{:03d}.jpg".format(dir_path, i)
            if os.path.isfile(image_path) and os.path.isfile(of_path):
                files.append(image_path)
                filesof.append("{}|{}".format(of_path, i-1))

        # append only the parts relative to the video section
        last = (len(files) - time_frames*skip) // skip
        rg = range(0, last + 1, np.max([1, int(stride * time_frames)]))
        for i in rg:
            temp, tpof = [], []
            for j in range(i*skip, (i+time_frames)*skip, skip):
                temp.append(files[j])
                tpof.append(filesof[j])
            videos_test.append(temp)
            opface_test.append(tpof)
            labels_test.append(Y[Yi])

            xl = []
            for l in range(Yi - 1, 0, -1):
                sub2 = clips[l].split(".")[0][:6]
                if subject == sub2:
                    xl.append(Y[l])
                if len(xl) == label_window:
                    break

            xl = [(xl[0] if len(xl) > 0 else np.array([0., 1.])) for x in range(label_window - len(xl))] + xl
            xlabel_test.append(xl)


    videos = [videos_test] + videos
    opface = [opface_test] + opface
    labels = [labels_test] + labels
    xlabel = [xlabel_test] + xlabel

    print("\nDONE")
    return videos, opface, xlabel, labels


def save_folds(folds, em):
    !mkdir folds

    first_only = False

    folds_copy = folds.copy()

    if len(folds) == 1:
        test_X = folds_copy[0][0].pop(0)
        test_XO = folds_copy[0][1].pop(0)
        test_XL = folds_copy[0][2].pop(0)
        test_Y = folds_copy[0][3].pop(0)

        videos = folds_copy[0][0]
        opface = folds_copy[0][1]
        xlabel = folds_copy[0][2]
        labels = folds_copy[0][3]

    elif len(folds) == 2:
        test_X = folds_copy[0][0].pop(0) + folds_copy[1][0].pop(0)
        test_XO = folds_copy[0][1].pop(0) + folds_copy[1][1].pop(0)
        test_XL = folds_copy[0][2].pop(0) + folds_copy[1][2].pop(0)
        test_Y = folds_copy[0][3].pop(0) + folds_copy[1][3].pop(0)

        videos = folds_copy[0][0] + folds_copy[1][0]
        opface = folds_copy[0][1] + folds_copy[1][1]
        xlabel = folds_copy[0][2] + folds_copy[1][2]
        labels = folds_copy[0][3] + folds_copy[1][3]

    test_Y = np.array(test_Y)
    print(test_Y.shape)
    np.save("folds/{}_test_Y.npy".format(em), test_Y)
    
    test_X = np.array(test_X)
    print(test_X.shape)
    np.save("folds/{}_test_XV.npy".format(em), test_X)

    test_XO = np.array(test_XO)
    print(test_XO.shape)
    np.save("folds/{}_test_XO.npy".format(em), test_XO)

    test_XL = np.array(test_XL)
    print(test_XL.shape)
    np.save("folds/{}_test_XL.npy".format(em), test_XL)

    for i in range(n_folds):
        print("SAVING FOLD {}...".format(i+1))
        train_X = videos.copy()
        train_XO = opface.copy()
        train_XL = xlabel.copy()
        train_Y = labels.copy()

        val_X = train_X.pop(i)
        val_XO = train_XO.pop(i)
        val_XL = train_XL.pop(i)
        val_Y = train_Y.pop(i)

        train_X = np.concatenate(train_X)
        train_XO = np.concatenate(train_XO)
        train_XL = np.concatenate(train_XL)
        train_Y = np.concatenate(train_Y)

        # create index array for resample
        I = np.array([x for x in range(len(train_X))]).reshape(-1,1)

        # I, train_Y = ros.fit_resample(I, train_Y)
        # I, train_Y = rus.fit_resample(I, train_Y)
        # train_Y = to_categorical(train_Y, len(original))

        # copy resampled elements from original arrays according to resampling indexes
        ntx, ntxo, ntxl = [], [], []
        for x in I:
            ntx.append(train_X[x[0]])
            ntxo.append(train_XO[x[0]])
            ntxl.append(train_XL[x[0]])
        train_X = ntx
        train_XO = ntxo
        train_XL = ntxl

        # save folds to file
        train_Y = np.array(train_Y)
        print(train_Y.shape)
        np.save("folds/{}_fold_{}_train_Y.npy".format(em, i), train_Y)
        
        train_X = np.array(train_X)
        print(train_X.shape)
        np.save("folds/{}_fold_{}_train_XV.npy".format(em, i), train_X)

        train_XO = np.array(train_XO)
        print(train_XO.shape)
        np.save("folds/{}_fold_{}_train_XO.npy".format(em, i), train_XO)

        train_XL = np.array(train_XL)
        print(train_XL.shape)
        np.save("folds/{}_fold_{}_train_XL.npy".format(em, i), train_XL)

        val_Y = np.array(val_Y)
        print(val_Y.shape)
        np.save("folds/{}_fold_{}_validation_Y.npy".format(em, i), val_Y)
        
        val_X = np.array(val_X)
        print(val_X.shape)
        np.save("folds/{}_fold_{}_validation_XV.npy".format(em, i), val_X)

        val_XO = np.array(val_XO)
        print(val_XO.shape)
        np.save("folds/{}_fold_{}_validation_XO.npy".format(em, i), val_XO)

        val_XL = np.array(val_XL)
        print(val_XL.shape)
        np.save("folds/{}_fold_{}_validation_XL.npy".format(em, i), val_XL)

        if first_only:
            break


class Generator_V(Sequence):
    def __init__(self, gen_split, batch_size, frames, emotion):
        print("BUILDING GENERATOR...")
        self.batch_size = batch_size 
        self.labels, self.videos, = [], []
        self.loadedOF = {}

        file_prefix = "" if gen_split == "Test" else "fold_{}_".format(fold_step-1)
        self.videos = np.load("folds/{}_{}{}_XV.npy".format(emotion, file_prefix, gen_split.lower()))
        self.opface = np.load("folds/{}_{}{}_XO.npy".format(emotion, file_prefix, gen_split.lower()))
        self.xlabel = np.load("folds/{}_{}{}_XL.npy".format(emotion, file_prefix, gen_split.lower()))
        self.labels = np.load("folds/{}_{}{}_Y.npy".format(emotion, file_prefix, gen_split.lower()))

        # self.videos, self.labels = shuffle(self.videos, self.labels, random_state=0)
        print("Done")


    def __len__(self):
        return int(np.ceil(len(self.videos) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_xv = self.videos[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_xo = self.opface[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_xl = self.xlabel[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        videos, opface = [], []
        for video in batch_xv:
            images = []
            for name in video:
                img = cv2.imread(name)
                img = cv2.resize(img, (224, 224))
                images.append(img/255)
            videos.append(np.array(images))

        for video in batch_xo:
            images = []

            of_path = video[0].split("|")[0]
            if not of_path in self.loadedOF:
                csv_of = pd.read_csv(of_path)
                rows = csv_of[[' gaze_0_x',' gaze_0_y',' gaze_0_z',' gaze_1_x',' gaze_1_y',' gaze_1_z',' gaze_angle_x',' gaze_angle_y',' pose_Tx',' pose_Ty',' pose_Tz',' pose_Rx',' pose_Ry',' pose_Rz',' AU01_r',' AU02_r',' AU04_r',' AU05_r',' AU06_r',' AU07_r',' AU09_r',' AU10_r',' AU12_r',' AU14_r',' AU15_r',' AU17_r',' AU20_r',' AU23_r',' AU25_r',' AU26_r',' AU45_r',' AU01_c',' AU02_c',' AU04_c',' AU05_c',' AU06_c',' AU07_c',' AU09_c',' AU10_c',' AU12_c',' AU14_c',' AU15_c',' AU17_c',' AU20_c',' AU23_c',' AU25_c',' AU26_c',' AU28_c',' AU45_c']]
                min_max_scaler = MinMaxScaler()
                self.loadedOF[of_path] = min_max_scaler.fit_transform(rows)

            rows = self.loadedOF[of_path]

            for name in video:
                i = int(name.split("|")[1])
                try:
                    img = rows[i]
                except:
                    print("ERROR", i, len(rows))
                images.append(img)

            opface.append(np.array(images))

        # videos = np.expand_dims(videos, axis=4)
        videos = np.array(videos)
        opface = np.array(opface)
        xlabel = np.array(batch_xl)
        label = np.array(batch_y)

        # regressor
        # label = np.array(batch_y).argmax(axis=1) / 3

        return [videos, opface, xlabel], label
        # return videos, label
        # return opface, label

def avgacc(y_true, y_pred):
    def get_class_acc(id):
        class_id_true = K.argmax(y_true, axis=-1)
        class_id_pred = K.argmax(y_pred, axis=-1)
        # Replace class_id_preds with class_id_true for recall here
        accuracy_mask = K.cast(K.equal(class_id_pred, id), 'int32')
        class_acc_tensor = K.cast(K.equal(class_id_true, class_id_pred), 'int32') * accuracy_mask
        class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
        return class_acc

    return sum([get_class_acc(0), get_class_acc(1)]) / 2


def build_model(epoch=0, print_model=False):
    from tcn import TCN, tcn_full_summary

    img_size = 224
    of_features = 49

    def model_convlstm(input):
        filters = 256

        x = ConvLSTM2D(filters, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(5e-4), recurrent_regularizer=l2(1e-6), return_sequences=True)(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = ConvLSTM2D(filters, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(5e-4), recurrent_regularizer=l2(1e-6), return_sequences=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = MaxPooling2D(pool_size=3, strides=2, padding='valid')(x)
        x = Flatten()(x)

        return x

    def model_openface(input):
        filters = 1024

        x = TCN(filters)(input)
        
        return x

    def model_inception(input):
        inception = tf.keras.applications.InceptionResNetV2(
            input_shape = (img_size, img_size, 3),
            weights = 'imagenet',
            include_top = False
        )

        for layer in inception.layers:
            layer.trainable = False

        x = TimeDistributed(inception)(input)

        return x

    print("Building model...")
    ########### IF BUILT, MUST DEFINE A NAME TO APPEND TO DIRECTORY NAME ###############
    global ident_name

    ########### IF LOADED, MUST DEFINE DIR NAME AND STARTING EPOCH ############
    global dir_name

    if not epoch:
        input_v = Input(shape=(time_frames, img_size, img_size, 3))
        input_o = Input(shape=(time_frames, of_features))
        input_l = Input(shape=(time_frames, 2))

        mv = model_inception(input_v)
        mv = model_convlstm(mv)
        mo = model_openface(input_o)
        ml = model_openface(input_l)

        # joint model
        x = Concatenate()([mv, mo, ml])
        
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.8)(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.8)(x)
        x = Dense(2, activation='softmax')(x)

        model = Model([input_v, input_o, input_l], x)
        # model = Model(input_v, x)

        model.compile(
            # loss='mse',
            loss='categorical_crossentropy',
            optimizer = SGD(learning_rate=0.0001, momentum=0.9), 
            # optimizer = Adam(learning_rate=1e-4),
            # metrics=[avgacc,'accuracy'])
            # metrics=['mse'])
            metrics=['accuracy'])
    
        t = datetime.datetime.now()
        prefix = str(t.year) +'-'+ str(t.month) +'-'+ str(t.day) +'-'+ str(t.hour) +'-'+ str(t.minute) +'-'+ str(t.second)
        save_dir = "{}/{}-{}".format(drive_save_path, prefix, ident_name)
        os.mkdir(save_dir)
    else:
        file_name = "{:03d}.h5".format(epoch)
        save_dir = "{}/{}".format(drive_save_path, dir_name)
        print("Loading model from {}/{}.".format(save_dir, file_name))
        model = load_model("{}/{}".format(save_dir, file_name), custom_objects={'TCN': TCN, 'avgacc': avgacc})

    if print_model:
        plot_model(model, show_layer_names=False, show_shapes=True, expand_nested=True)
        display(Image('model.png'))

    return model, save_dir, epoch


def set_callbacks():
    #callbacks
    checkpoint = ModelCheckpoint(
        filepath = save_dir + '/{epoch:03d}.h5', 
        monitor = 'val_loss', 
        verbose=1, 
        save_best_only=True,
    )

    tensorboard = TensorBoard(
    	log_dir         = "{}/logs".format(save_dir),
    	histogram_freq  = 0,
    	write_graph     = True,
    	write_grads     = False,
    	write_images    = True
    )

    early_stop = EarlyStopping(
        monitor 	= 'val_loss',
        patience 	= 20,
        restore_best_weights = True,
        verbose     = 1,
        min_delta   = 1e-5
    )

    reduce_lr_plateau = ReduceLROnPlateau(
        monitor 	= 'val_loss',
        factor		= 0.5,
        patience	= 10,
        min_lr		= 1e-6,
        verbose     = 1
    )

    # return [checkpoint, tensorboard]
    return [checkpoint, early_stop, reduce_lr_plateau, tensorboard]


def fit_model(train_weights, epoch=0):
    #calculate weights based on train set distribution
    num_classes = len(train_weights)
    if train_weights == [1 for x in range(num_classes)]:
        weights = {x:1 for x in range(num_classes)}
    else:
        weights = {x: train_weights[x] for x in range(len(train_weights))}
        print("Train weights: {}".format(weights))

    def run():
        #run the model
        return model.fit(
            gen_train,
            epochs = 1000,
            validation_data = gen_val,
            class_weight = weights,
            callbacks = callbacks,
            initial_epoch = epoch)
        
    hist = run()  

    plot_hist(hist)

    return hist


def plot_hist(hist):
    plt.plot(hist.history['loss'], '#0000ff', label="loss")
    plt.plot(hist.history['val_loss'], '#ff0000', label="val_loss")
    plt.legend()
    plt.show()

    plt.plot(hist.history['accuracy'], '#0000ff', label="acc")
    plt.plot(hist.history['val_accuracy'], '#ff0000', label="val_acc")
    # plt.plot(hist.history['accuracy'], '#0055aa', label="acc")
    # plt.plot(hist.history['val_accuracy'], '#aa5500', label="val_acc")
    plt.legend()
    plt.show()


def predict(**kw):
    global gen_test

    Y_true = np.load("folds/{}_test_Y.npy".format(emotion))
    num_classes = Y_true.shape[1]
    Y_true = Y_true.argmax(axis=1)

    print("Predicting...")
    Y_pred = model.predict(gen_test, verbose=1).argmax(axis=1)
    print("\nConfusion matrix:")
    cm = confusion_matrix(Y_true, Y_pred)
    print(cm)

    print("\nEvaluating...")
    ev = model.evaluate(gen_test, verbose=1)
    # print("Loss: {}, Acc: {}".format(ev[0], ev[1]))

    hits = [0 for x in range(num_classes)]
    total = [0 for x in range(num_classes)]
    for i in range(len(Y_true)):
        for c in range(num_classes):
            if (Y_true[i] == c and Y_pred[i] == c):
                hits[c] = hits[c] + 1
            if Y_true[i] == c:
                total[c] = total[c] + 1

    print("\nPer class accuracy:")
    for c in range(num_classes):
        hits[c] = hits[c] / total[c]
        print("{}: {}".format(c, hits[c]))

    print("Average class accuracy: {}".format(sum(hits) / len(hits)))

    f1 = f1_score(Y_true, Y_pred, average=None)
    print("F1 score: {}. Avg F1: {}".format(f1, np.mean(f1)))

    return gen_test
        

In [ ]:
# IF EVER YOU NEED TO RECREATE THE FOLDS
# emotionList = ["engagement", "confusion", "frustration", "boredom"]
# for em in emotionList:
#     fold_p2m = create_folds_p2m(em)
#     fold_daisee = create_folds_daisee(em)
#     save_folds([fold_p2m, fold_daisee], em)

# !rm -f /content/drive/MyDrive/1NOSYNC/DT/folds.zip
# !zip /content/drive/MyDrive/1NOSYNC/DT/folds.zip folds/*

In [ ]:
# start_colab()
# extract_data()
# !unzip /content/drive/MyDrive/1NOSYNC/DT/folds.zip -d ./

# fold_step = 1
# emotion = 'frustration'
# gen_train = Generator_V('Train', batch_size, time_frames, emotion)
# gen_val = Generator_V('Validation', batch_size, time_frames, emotion)
# gen_test = Generator_V('Test', batch_size, time_frames, emotion)

# dir_name = '2022-8-6-14-4-29-final'
# epoch = 25
model, save_dir, epoch = build_model(epoch, print_model=True)

In [ ]:
# callbacks = set_callbacks()
# fit_model(list(len(gen_train.labels) / np.sum(gen_train.labels, axis=0)), epoch)
gen_test = predict()